# Jour 2 - Exercice 1 : Feature Engineering et Transformation des données

## Objectifs
- Comprendre l'importance du feature engineering dans le machine learning
- Apprendre à transformer les variables catégorielles en variables numériques
- Standardiser et normaliser les données numériques
- Créer de nouvelles features à partir des données existantes
- Préparer un pipeline de prétraitement pour le machine learning

## Introduction

Le feature engineering est une étape cruciale dans tout projet de machine learning. Il s'agit de transformer les données brutes en features (caractéristiques) qui peuvent être utilisées efficacement par les algorithmes de ML. Dans ce notebook, nous allons explorer différentes techniques de feature engineering et de transformation des données en utilisant le jeu de données de satisfaction des passagers.

## 1. Chargement et exploration initiale des données

In [1]:
# Importation des bibliothèques
import pandas as pd
import numpy as np
import seaborn as sns
import plotly
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Pour afficher plus de colonnes dans les DataFrames
pd.set_option('display.max_columns', 30)

In [ ]:
# Chargement du jeu de données
df = pd.read_csv('../../data/passenger_satisfaction/train.csv')

# Affichage des premières lignes
df.head()

In [ ]:
# Informations sur le DataFrame
df.info()

In [ ]:
# Statistiques descriptives
df.describe()

## 2. Traitement des valeurs manquantes

Avant de commencer le feature engineering, il est important de traiter les valeurs manquantes dans notre jeu de données.

In [ ]:
# Vérification des valeurs manquantes
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

# Création d'un DataFrame pour afficher les résultats
missing_df = pd.DataFrame({
    'Nombre de valeurs manquantes': missing_values,
    'Pourcentage (%)': missing_percentage
})

# Affichage des colonnes avec des valeurs manquantes
missing_df[missing_df['Nombre de valeurs manquantes'] > 0].sort_values('Nombre de valeurs manquantes', ascending=False)

### 2.1 Visualisation des valeurs manquantes

In [ ]:
# Visualisation des valeurs manquantes avec Plotly
missing_cols = missing_df[missing_df['Nombre de valeurs manquantes'] > 0].index.tolist()
missing_values = missing_df.loc[missing_cols, 'Pourcentage (%)'].values

fig = px.bar(
    x=missing_cols,
    y=missing_values,
    labels={'x': 'Colonnes', 'y': 'Pourcentage de valeurs manquantes (%)'},
    title='Pourcentage de valeurs manquantes par colonne',
    color=missing_values,
    color_continuous_scale='Viridis'
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

### 2.2 Stratégie pour traiter les valeurs manquantes

Nous avons plusieurs options pour traiter les valeurs manquantes :
1. Supprimer les lignes avec des valeurs manquantes
2. Remplacer les valeurs manquantes par une valeur par défaut (moyenne, médiane, mode, etc.)
3. Utiliser des techniques plus avancées comme l'imputation par KNN ou par modèles prédictifs

Pour cet exercice, nous allons utiliser une approche simple mais efficace :

In [ ]:
# Copie du DataFrame pour ne pas modifier l'original
df_clean = df.copy()

# Pour les variables numériques, remplacer les valeurs manquantes par la médiane
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col].fillna(df_clean[col].median(), inplace=True)

# Pour les variables catégorielles, remplacer les valeurs manquantes par le mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df_clean[col].isnull().sum() > 0:
        df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)

# Vérification que toutes les valeurs manquantes ont été traitées
print("Nombre total de valeurs manquantes après traitement :", df_clean.isnull().sum().sum())

## 3. Transformation des variables catégorielles

Les algorithmes de machine learning ne peuvent pas traiter directement les variables catégorielles (texte). Nous devons les convertir en variables numériques.

### 3.1 Identification des variables catégorielles

In [ ]:
# Affichage des variables catégorielles et de leurs valeurs uniques
for col in categorical_cols:
    print(f"\nColonne: {col}")
    print(f"Nombre de valeurs uniques: {df_clean[col].nunique()}")
    print(f"Valeurs uniques: {df_clean[col].unique()}")

### 3.2 Encodage des variables catégorielles

Il existe plusieurs méthodes pour encoder les variables catégorielles :
1. **Label Encoding** : Attribue un nombre entier unique à chaque catégorie (0, 1, 2, etc.)
2. **One-Hot Encoding** : Crée une nouvelle colonne binaire pour chaque catégorie
3. **Target Encoding** : Remplace chaque catégorie par la moyenne de la variable cible pour cette catégorie

Choisissons la méthode appropriée pour chaque variable :

In [ ]:
# Copie du DataFrame pour les transformations
df_encoded = df_clean.copy()

# Label Encoding pour les variables ordinales ou binaires
label_encode_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'Satisfaction']

for col in label_encode_cols:
    le = LabelEncoder()
    df_encoded[col + '_encoded'] = le.fit_transform(df_encoded[col])
    # Afficher la correspondance entre les valeurs originales et encodées
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    print(f"\nEncodage pour {col}:")
    for original, encoded in mapping.items():
        print(f"  {original} -> {encoded}")

In [ ]:
# Ordinal Encoding pour la variable Class qui a un ordre naturel
# Eco < Eco Plus < Business
class_mapping = {
    'Eco': 0,
    'Eco Plus': 1, 
    'Business': 2
}

# Application de l'encodage ordinal
df_encoded['Class_ordinal'] = df_encoded['Class'].map(class_mapping)

# Affichage de la correspondance
print("Encodage ordinal pour Class:")
for classe, valeur in class_mapping.items():
    print(f"  {classe} -> {valeur}")

In [ ]:
# Affichage des premières lignes du DataFrame transformé
df_encoded.head()

## 4. Standardisation et normalisation des variables numériques

De nombreux algorithmes de machine learning sont sensibles à l'échelle des variables. Il est souvent recommandé de standardiser ou normaliser les variables numériques.

### 4.1 Identification des variables numériques à transformer

In [ ]:
# Sélection des colonnes numériques (en excluant l'ID et les colonnes déjà encodées)
numeric_cols_to_scale = [col for col in numeric_cols if col != 'id' and not col.endswith('_encoded')]
print("Colonnes numériques à standardiser/normaliser :")
print(numeric_cols_to_scale)

### 4.2 Standardisation (Z-score)

La standardisation transforme les données pour qu'elles aient une moyenne de 0 et un écart-type de 1.

In [ ]:
# Copie du DataFrame pour la standardisation
df_standardized = df_encoded.copy()

# Standardisation des variables numériques
scaler = StandardScaler()
df_standardized[numeric_cols_to_scale] = scaler.fit_transform(df_standardized[numeric_cols_to_scale])

# Affichage des premières lignes des colonnes standardisées
df_standardized[numeric_cols_to_scale].head()

### 4.3 Normalisation (Min-Max)

La normalisation transforme les données pour qu'elles soient dans un intervalle spécifique, généralement [0, 1].

In [ ]:
# Copie du DataFrame pour la normalisation
df_normalized = df_encoded.copy()

# Normalisation des variables numériques
normalizer = MinMaxScaler()
df_normalized[numeric_cols_to_scale] = normalizer.fit_transform(df_normalized[numeric_cols_to_scale])

# Affichage des premières lignes des colonnes normalisées
df_normalized[numeric_cols_to_scale].head()

### 4.4 Comparaison des distributions avant et après transformation

In [ ]:
# Sélection de quelques colonnes numériques pour la visualisation
cols_to_plot = ['Age', 'Flight Distance', 'Departure Delay in Minutes']
titles = ['Original', 'Standardisé', 'Normalisé']
dfs = [df_clean, df_standardized, df_normalized]

# Création des subplots avec plotly
fig = make_subplots(rows=3, cols=3, 
                    subplot_titles=[f'{title}: {col}' for col in cols_to_plot for title in titles])

for i, col in enumerate(cols_to_plot):
    for j, (df, title) in enumerate(zip(dfs, titles)):
        fig.add_trace(
            go.Histogram(x=df[col], name=f'{title} {col}', 
                        nbinsx=30, histnorm='probability density'),
            row=i+1, col=j+1
        )

fig.update_layout(
    height=800,
    width=1200,
    title_text='Comparaison des distributions avant et après transformation',
    showlegend=False
)

fig.show()

## 5. Création de nouvelles features

Le feature engineering ne se limite pas à transformer les variables existantes. Nous pouvons également créer de nouvelles features qui pourraient être utiles pour notre modèle.

### 5.1 Création de features basées sur des connaissances du domaine

In [ ]:
# Copie du DataFrame pour la création de nouvelles features
df_features = df_clean.copy()

# Calcul du retard total (départ + arrivée)
df_features['Total Delay'] = df_features['Departure Delay in Minutes'] + df_features['Arrival Delay in Minutes']

# Création d'une feature indiquant si le vol a été retardé
df_features['Is Delayed'] = (df_features['Departure Delay in Minutes'] > 0).astype(int)

# Création d'une feature pour la satisfaction moyenne des services
service_cols = [
    'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking',
    'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service',
    'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness'
]
df_features['Average Service Rating'] = df_features[service_cols].mean(axis=1)

# Création d'une feature pour le nombre de services mal notés (note < 3)
df_features['Poor Services Count'] = (df_features[service_cols] < 3).sum(axis=1)

# Affichage des nouvelles features
df_features[['Total Delay', 'Is Delayed', 'Average Service Rating', 'Poor Services Count']].head()

### 5.2 Analyse de l'importance des nouvelles features

In [ ]:
# Visualisation de la relation entre les nouvelles features et la satisfaction
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Retard total vs Satisfaction',
        'Vol retardé vs Satisfaction', 
        'Note moyenne des services vs Satisfaction',
        'Nombre de services mal notés vs Satisfaction'
    )
)

# Total Delay vs Satisfaction
fig.add_trace(
    go.Box(x=df_features['Satisfaction'], y=df_features['Total Delay'], name='Total Delay'),
    row=1, col=1
)

# Is Delayed vs Satisfaction
delayed_counts = df_features.groupby(['Is Delayed', 'Satisfaction']).size().reset_index(name='count')
fig.add_trace(
    go.Bar(x=delayed_counts['Is Delayed'], y=delayed_counts['count'], 
           marker_color=['#1f77b4' if x == 'satisfied' else '#ff7f0e' for x in delayed_counts['Satisfaction']], name='Is Delayed'),
    row=1, col=2
)

# Average Service Rating vs Satisfaction
fig.add_trace(
    go.Box(x=df_features['Satisfaction'], y=df_features['Average Service Rating'], 
           name='Average Service Rating'),
    row=2, col=1
)

# Poor Services Count vs Satisfaction
fig.add_trace(
    go.Box(x=df_features['Satisfaction'], y=df_features['Poor Services Count'], 
           name='Poor Services Count'),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title_text='Relation entre les nouvelles features et la satisfaction',
    height=800,
    showlegend=False
)

fig.show()

## 6. Création d'un pipeline de prétraitement

Pour automatiser le processus de prétraitement des données, nous pouvons créer un pipeline avec scikit-learn. Cela nous permettra d'appliquer facilement les mêmes transformations à de nouvelles données.

In [ ]:
# Définition des colonnes pour chaque type de transformation
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
service_cols = [
    'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking',
    'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service',
    'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness'
]

# Création du pipeline de prétraitement
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numerical_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols),
        ('serv', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), service_cols)
    ],
    remainder='drop'  # Ignorer les colonnes qui ne sont pas spécifiées
)

# Affichage du pipeline
print(preprocessor)

In [ ]:
# Application du pipeline sur les données
X = df.drop(['ID', 'Satisfaction'], axis=1)  # Features
y = df['Satisfaction']  # Variable cible

# Transformation des données
X_transformed = preprocessor.fit_transform(X)

# Affichage de la forme des données transformées
print(f"Forme des données transformées: {X_transformed.shape}")

## 7. Exercices pratiques

Maintenant que nous avons exploré différentes techniques de feature engineering, mettons en pratique ces connaissances avec quelques exercices.

### Exercice 1: Création de nouvelles features

Créez au moins deux nouvelles features qui pourraient être utiles pour prédire la satisfaction des passagers. Justifiez votre choix et analysez leur relation avec la variable cible.

In [ ]:
# Votre code ici
# Exemple:
df_ex1 = df_clean.copy()

# Feature 1: Ratio entre la distance du vol et le retard total
# Justification: Un petit retard sur un vol court peut être plus frustrant qu'un retard similaire sur un vol long
df_ex1['Delay_Distance_Ratio'] = (df_ex1['Departure Delay in Minutes'] + df_ex1['Arrival Delay in Minutes']) / (df_ex1['Flight Distance'] + 1)  # +1 pour éviter division par zéro

# Feature 2: Score de confort global (combinaison de plusieurs critères liés au confort)
# Justification: Le confort global peut être un facteur déterminant de la satisfaction
comfort_cols = ['Seat comfort', 'Leg room service', 'Cleanliness', 'Food and drink', 'Inflight entertainment']
df_ex1['Comfort_Score'] = df_ex1[comfort_cols].mean(axis=1)

# Création des box plots avec plotly
fig = make_subplots(rows=1, cols=2, subplot_titles=('Ratio Retard/Distance vs Satisfaction', 
                                                   'Score de confort vs Satisfaction'))

# Delay_Distance_Ratio vs Satisfaction
fig.add_trace(
    go.Box(x=df_ex1['Satisfaction'], y=df_ex1['Delay_Distance_Ratio'], name='Delay/Distance'),
    row=1, col=1
)

# Comfort_Score vs Satisfaction
fig.add_trace(
    go.Box(x=df_ex1['Satisfaction'], y=df_ex1['Comfort_Score'], name='Comfort'),
    row=1, col=2
)

# Mise à jour du layout
fig.update_layout(
    title_text='Relation entre les nouvelles features et la satisfaction',
    showlegend=False,
    height=600,
    width=1200
)

# Échelle logarithmique pour le premier graphique
fig.update_yaxes(type="log", row=1, col=1)

# Affichage du graphique
fig.show()

### Exercice 2: Comparaison des méthodes d'encodage

Comparez l'impact de différentes méthodes d'encodage (Label Encoding vs One-Hot Encoding) sur une variable catégorielle de votre choix. Discutez des avantages et inconvénients de chaque méthode.

In [ ]:
# Votre code ici
# Exemple avec la variable 'Class'
df_ex2 = df_clean.copy()

# Label Encoding
le = LabelEncoder()
df_ex2['Class_Label_Encoded'] = le.fit_transform(df_ex2['Class'])
print("Label Encoding pour 'Class':")
for original, encoded in zip(le.classes_, le.transform(le.classes_)):
    print(f"  {original} -> {encoded}")

# One-Hot Encoding
one_hot = pd.get_dummies(df_ex2['Class'], prefix='Class')
df_ex2 = pd.concat([df_ex2, one_hot], axis=1)
print("\nOne-Hot Encoding pour 'Class':")
print(one_hot.head())

# Visualisation avec plotly
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Label Encoding', 'One-Hot Encoding'),
                    specs=[[{"type": "bar"}, {"type": "bar"}]])

# Distribution de la variable avec Label Encoding
label_counts = df_ex2['Class_Label_Encoded'].value_counts().sort_index()
fig.add_trace(
    go.Bar(x=label_counts.index, y=label_counts.values, name='Label Encoding'),
    row=1, col=1
)

# Distribution de la variable avec One-Hot Encoding
fig.add_trace(
    go.Bar(x=one_hot.columns, y=one_hot.sum(), name='One-Hot Encoding'),
    row=1, col=2
)

# Mise à jour du layout
fig.update_layout(
    title_text='Comparaison des méthodes d\'encodage pour la variable Class',
    showlegend=False,
    height=600,
    width=1200
)

# Mise à jour des axes
fig.update_xaxes(title_text='Class (encodée)', row=1, col=1)
fig.update_xaxes(title_text='Class (one-hot)', row=1, col=2)
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Count', row=1, col=2)

# Affichage du graphique
fig.show()

# Discussion
print("\nDiscussion:")
print("Label Encoding:")
print("  Avantages: Simple, conserve une seule colonne, peut capturer l'ordre si la variable est ordinale")
print("  Inconvénients: Introduit une relation d'ordre artificielle pour les variables nominales")
print("\nOne-Hot Encoding:")
print("  Avantages: Pas de relation d'ordre artificielle, meilleur pour les variables nominales")
print("  Inconvénients: Augmente le nombre de colonnes, peut créer de la multicolinéarité")

### Exercice 3: Création d'un pipeline personnalisé

Créez un pipeline personnalisé qui combine plusieurs étapes de prétraitement, y compris la création de nouvelles features, l'encodage des variables catégorielles et la standardisation des variables numériques.

In [ ]:
# Votre code ici
from sklearn.base import BaseEstimator, TransformerMixin

# Création d'un transformateur personnalisé pour la création de features
class FeatureCreator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        
        # Calcul du retard total
        X_copy['Total_Delay'] = X_copy['Departure Delay in Minutes'] + X_copy['Arrival Delay in Minutes']
        
        # Score de confort
        comfort_cols = ['Seat comfort', 'Leg room service', 'Cleanliness', 'Food and drink', 'Inflight entertainment']
        X_copy['Comfort_Score'] = X_copy[comfort_cols].mean(axis=1)
        
        # Score de service
        service_cols = ['Inflight wifi service', 'On-board service', 'Baggage handling', 'Checkin service', 'Inflight service']
        X_copy['Service_Score'] = X_copy[service_cols].mean(axis=1)
        
        return X_copy

# Création du pipeline personnalisé
from sklearn.pipeline import Pipeline

# Définition des colonnes
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
service_cols = [
    'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking',
    'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
    'Inflight entertainment', 'On-board service', 'Leg room service',
    'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness'
]

# Colonnes à conserver pour la création de features
cols_to_keep = numerical_cols + categorical_cols + service_cols

# Pipeline complet
custom_pipeline = Pipeline([
    ('feature_creator', FeatureCreator()),
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), numerical_cols + ['Total_Delay', 'Comfort_Score', 'Service_Score']),
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))
            ]), categorical_cols),
            ('serv', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), service_cols)
        ],
        remainder='drop'
    ))
])

# Test du pipeline
X = df.drop(['ID', 'Satisfaction'], axis=1)  # Features
y = df['Satisfaction']  # Variable cible

# Application du pipeline
X_transformed = custom_pipeline.fit_transform(X)
print(f"Forme des données transformées avec le pipeline personnalisé: {X_transformed.shape}")

## 10. Conclusion

Dans ce notebook, nous avons exploré différentes techniques de feature engineering et de transformation des données :

1. **Traitement des valeurs manquantes** : Nous avons identifié et traité les valeurs manquantes dans notre jeu de données.
2. **Transformation des variables catégorielles** : Nous avons utilisé le Label Encoding et le One-Hot Encoding pour convertir les variables catégorielles en variables numériques.
3. **Standardisation et normalisation** : Nous avons appliqué différentes techniques pour mettre à l'échelle les variables numériques.
4. **Création de nouvelles features** : Nous avons créé de nouvelles features à partir des données existantes pour améliorer la performance des modèles.
5. **Création d'un pipeline de prétraitement** : Nous avons automatisé le processus de prétraitement des données avec un pipeline scikit-learn.

Ces techniques sont essentielles pour préparer les données avant d'entraîner des modèles de machine learning. Dans le prochain notebook, nous utiliserons ces données prétraitées pour entraîner nos premiers modèles de classification.